# Выделение ключевых слов

__Идея:__ я хочу проверить, насколько хорошо будет работать матрица совстречаемости, которая будет основана на полном проходе по всему тексту (а не случайному выбору, пусть и большое количество раз, как в random walk, который мы обсуждали на парах).

_Дополнительная идея:_ посмотреть, как результат будет зависеть от величины «окна», которую мы зададим.

## Алгоритм

### Обработка текста

In [1]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [2]:
mystem = Mystem()
stops = stopwords.words("russian")
punctuators = punctuation + "«–»— \r\n"

In [3]:
def lemmatize_text(text):
    """Из исходного текста делает список лемм, очищенных от пунктуации
    и стоп-слов.
    
    :arg text (str): исходный текст, как он загружен из файла
    :returns clean_text (list of str): список лемм текста в том порядке, в
    котором они были в тексте
    """
    lemmas_raw = [parse["analysis"][0]["lex"] for parse in mystem.analyze(text)
                  if parse.get("analysis")]
    clean_text = [lemma_raw for lemma_raw in lemmas_raw
                 if lemma_raw not in stops and lemma_raw not in punctuators]        
    return clean_text

### Составление матрицы смежности

In [4]:
import numpy as np
from heapq import nlargest

In [5]:
def fill_co_occurence_matrix(text_lemmas, word_list, window):
    """Собирает матрицу совстречаемости из текста.
    
    :arg text_lemmas (list of str): лемматизированный и почищенный текст
    :arg word_list (list of str): список уникальных слов
    :arg window (int): окно, в котором мы считаем совстречаемость
    
    :returns M_co_occur (np.ndarray): матрица совстречаемости
    """
    M_co_occur = np.zeros((len(word_list), len(word_list)))
    for lemma_ind, lemma in enumerate(text_lemmas):
        lemma_matrix_index = word_list.index(lemma)
        ind_left = max(0, lemma_ind - window)
        ind_right = min(lemma_ind + window, len(text_lemmas))
        for ind_neighbor in range(ind_left, ind_right):
            word_neighbor = text_lemmas[ind_neighbor]
            neighbor_matrix_index = word_list.index(word_neighbor)
            M_co_occur[lemma_matrix_index][neighbor_matrix_index] += 1
    return M_co_occur

Предположим, что ключевыми будут те слова, которые соединяются с наибольшим количеством других слов => их совстречаемость будет самой высокой.

<font style="color: blue">**UPD 6.05:** исправила баг и поменяла `argmax` на `sum`.
Совстречаемые слова теперь подбираются с помощью выбора `max_items` наибольших на векторе сумм встречаемостей (алгоритм [тут](https://medium.com/@yurybelousov/the-beauty-of-python-or-how-to-get-indices-of-n-maximum-values-in-the-array-d362385794ef)).
</font>

In [70]:
def extract_keywords_from_matrix(matrix, word_list, max_items):
    """Из матрицы совстречаемости выбирает сколько-то слов с наибольшей 
    совстречаемостью.
    
    :arg matrix (np.ndarray): матрица совстречаемости
    :arg word_list (list of str): список уникальных слов, на котором строилась 
    матрица
    :arg max_items (int): сколько первых значений совстречаемости брать
    
    :returns keywords (list of str): сами ключевые слова
    """
    matrix_sum = matrix.sum(axis=1)
    # выглядит страшно, но просто это оптимальный алгоритм для поиска N наибольших величин
    # реализован через очередь со стеками, работает за линейное время — для заметок, может, не так
    # уж и важно, но для больших текстов уже будет заметно
    kw_indices = nlargest(max_items, range(len(matrix_sum)), matrix_sum.__getitem__)
    keywords = [word_list[kw_ind] for kw_ind in kw_indices]
    return keywords

### Измеряем качество

In [7]:
def evaluate_kws(kws_true, kws_predicted):
    """Считает точность, полноту, F-меру и коэффициент Жаккара.
    
    :arg kws_true (list of str): размеченные вручную ключевые слова
    :arg kws_predicted (list of str): ключевые слова, найденные алгоритмом
    
    :returns precision, recall, f_score, jaccard (float): метрики
    """
    intersection = len(set(kws_true) & set(kws_predicted))
    precision = intersection / len(kws_predicted)
    recall = intersection / len(kws_true)
    try:
        f_score = (2 * precision * recall)/(precision + recall)
    except:
        f_score = 0
    jaccard = intersection / len(set(kws_true) | set(kws_predicted))
    return precision, recall, f_score, jaccard

### Всё вместе

In [77]:
def pipeline_for_texts(text, kw_true):
    """Единая функция-обёртка для работы с текстом от и до.
    
    :arg text (str): исходный текст
    :arg kw_true (list of str): приписанные вручную ключевые слова из датасета
    """
    text_lemmas = lemmatize_text(text)
    word_list = list(set(text_lemmas))
    for window in range(1, 6):
        print("window={}".format(window))
        M = fill_co_occurence_matrix(text_lemmas, word_list, window)
        for thresh in range(1, 11):
            kw = extract_keywords_from_matrix(M, word_list, thresh)
            print("\ttop-{} most co-occurring words: {}".format(thresh, kw))
            precision, recall, f1, jaccard = evaluate_kws(kw_true, kw)
            print("\tP={:.5f}, R={:.5f}, F1={:.5f}, J={:.5f}".format(precision, recall, f1, jaccard))

## Подгружаем данные из датасета

Датасет — [ru-kw-eval](https://github.com/mannefedov/ru_kw_eval_datasets), я взяла одну из Независимых газет (ха-ха).

In [9]:
import json

In [10]:
ng_data = []
with open("../data/ng_0.jsonlines", "r") as f:
    for line in f.readlines():
        ng_data.append(json.loads(line))

Возьмём какую-нибудь случайную статью и посмотрим, как работает:

In [11]:
import random

<font style="color: blue">**UPD 6.05:** Скопируйте код отсюда, чтобы это был действительно случайный эксперимент: </font>

```python
random.seed() 
article_ind = random.choice(range(0, len(ng_data)-1))
article_data = ng_data[article_ind]

print("TITLE: {}".format(article_data["title"]))
print("KEYWORDS: {}".format(article_data["keywords"]))

pipeline_for_texts(article_data["content"], article_data["keywords"])
```

<font style="color: blue">Я уже наигралась, да и вообще это доделка, поэтому внизу будут два варианта с хардкодом — один получше, другой похуже.</font>

In [78]:
worse_res_ind = 437
# Москва объявила всем,  что обладает принципиально новым сверхмощным оружием
article_data = ng_data[worse_res_ind]

print("TITLE: {}".format(article_data["title"]))
print("KEYWORDS: {}".format(article_data["keywords"]))

pipeline_for_texts(article_data["content"], article_data["keywords"])

TITLE: Москва объявила всем,  что обладает принципиально новым сверхмощным оружием
KEYWORDS: ['президент', 'послание', 'вооружения', 'стратегическое оружие']
window=1
	top-1 most co-occurring words: ['ракета']
	P=0.00000, R=0.00000, F1=0.00000, J=0.00000
	top-2 most co-occurring words: ['ракета', 'новый']
	P=0.00000, R=0.00000, F1=0.00000, J=0.00000
	top-3 most co-occurring words: ['ракета', 'новый', 'комплекс']
	P=0.00000, R=0.00000, F1=0.00000, J=0.00000
	top-4 most co-occurring words: ['ракета', 'новый', 'комплекс', 'система']
	P=0.00000, R=0.00000, F1=0.00000, J=0.00000
	top-5 most co-occurring words: ['ракета', 'новый', 'комплекс', 'система', 'президент']
	P=0.20000, R=0.25000, F1=0.22222, J=0.12500
	top-6 most co-occurring words: ['ракета', 'новый', 'комплекс', 'система', 'президент', 'владимир']
	P=0.16667, R=0.25000, F1=0.20000, J=0.11111
	top-7 most co-occurring words: ['ракета', 'новый', 'комплекс', 'система', 'президент', 'владимир', 'ракетный']
	P=0.14286, R=0.25000, F1=0.1

In [95]:
better_res_ind = 827
# "Талибан" берет верх в Афганистане
article_data = ng_data[better_res_ind]

print("TITLE: {}".format(article_data["title"]))
print("KEYWORDS: {}".format(article_data["keywords"]))

pipeline_for_texts(article_data["content"], article_data["keywords"])

TITLE: "Талибан" берет верх в Афганистане
KEYWORDS: ['афганистан', 'талибан', 'даиш']
window=1
	top-1 most co-occurring words: ['даиш']
	P=1.00000, R=0.33333, F1=0.50000, J=0.33333
	top-2 most co-occurring words: ['даиш', 'группа']
	P=0.50000, R=0.33333, F1=0.40000, J=0.25000
	top-3 most co-occurring words: ['даиш', 'группа', 'шура']
	P=0.33333, R=0.33333, F1=0.33333, J=0.20000
	top-4 most co-occurring words: ['даиш', 'группа', 'шура', 'афганистан']
	P=0.50000, R=0.66667, F1=0.57143, J=0.40000
	top-5 most co-occurring words: ['даиш', 'группа', 'шура', 'афганистан', 'кветта']
	P=0.40000, R=0.66667, F1=0.50000, J=0.33333
	top-6 most co-occurring words: ['даиш', 'группа', 'шура', 'афганистан', 'кветта', 'год']
	P=0.33333, R=0.66667, F1=0.44444, J=0.28571
	top-7 most co-occurring words: ['даиш', 'группа', 'шура', 'афганистан', 'кветта', 'год', 'фарука']
	P=0.28571, R=0.66667, F1=0.40000, J=0.25000
	top-8 most co-occurring words: ['даиш', 'группа', 'шура', 'афганистан', 'кветта', 'год', 'фа

## Выводы

1) Возможно, результаты такие ужасные, потому что в ключевых словах есть ещё и биграммы, которые не реализованы в этом алгоритме;

2) random walk не такой уж и плохой алгоритм;

3) новостные заметки небольшие, и поэтому совстречаемость может работать не очень хорошо, т.к. на малом объёме будет большая вариативность лемм, а чем текст больше, тем больше леммы повторяются.

<font style="color: blue">**UPD 6.05:**
    
1) Ну, самое топовое слово — нередко ключевое, `precision=1` ужасно радует глаз (но происходит это не всегда — я случайно позапускала последнюю ячейку с кодом и в 50% случаев попадала на такой результат.
    
2) Зависимость F-меры от «окна» и «количества» слов отследить пока не очень могу — наверное, можно визуализировать, чтобы совсем хорошо жилось, но <s>после перебора кучи статей</s> интуитивно кажется, что нужно брать 2-3 слова с небольшим окном (не больше трёх).
</font>